# Basics

Mostly exercises and experiments made during
[Martin Odersky's functional programming course on Coursera](https://www.coursera.org/learn/progfun1)

## (Tail) recursion

In [3]:
import scala.annotation.tailrec

def fibonacci_tail_recursive(n: Long): BigInt = {
// @tailrec enforces method it annotates to be tail recursive
@tailrec def fib(n: Long, acc: BigInt): BigInt = n match {
  case n if n >= 1 => fib(n - 1, acc * n)
  case _ => acc
  }

  fib(n, 1)
}

def fibonacci_not_tail_recursive(n: BigInt): BigInt = n match {
  case n if n > 1 => n * fibonacci_not_tail_recursive(n - 1)
  case _ => 1
}

fibonacci_tail_recursive(100)

fibonacci_not_tail_recursive(100) // should work if currect max stack size is > 1000

fibonacci_tail_recursive(50000)

// fibonacci_not_tail_recursive(50000) // will probably not work without changing jvm settings

try {
  (1000 to 20000 by 1000).map { i =>
    println(s"I is: $i")
    try {
      fibonacci_not_tail_recursive(i)
    } catch {
      case e: StackOverflowError =>
        throw new RuntimeException(
          s"""
             |Stack length that we reached was $i
             |see https://blogs.oracle.com/saas-fusion-app-performance/how-to-set-stack-size-to-overcome-javalangstackoverflowerror")
           """.stripMargin)
    }
  }
} catch {
  case e: RuntimeException =>
    println(e.getMessage)
}

def sum(i: Int, j: Int, f: Int => Int): Int = {
  if (i < j) f(i) + sum(i + 1, j, f)
  else f(j)
}

def sumtr(i: Int, j: Int, f: Int => Int): Int = {
  @tailrec def loop(acc: Int, i: Int, j: Int, f: Int => Int): Int = {
    if(i > j) acc
    else loop(acc + f(i), i + 1, j, f)
  }

  loop(0, i, j, f)
}
sumtr(1, 3, k => k * k)
sum(1, 3, k => k * k)

val params = Tuple3(1,5,(k:Int) => k * k * k)

assert(sum(1, 3, k => k * k) == sumtr(1, 3, k => k * k))
assert(sum(1,5, k => k * k * k) == sumtr(params._1, params._2, params._3))


I is: 1000
I is: 2000
I is: 3000
I is: 4000
I is: 5000

Stack length that we reached was 5000
see https://blogs.oracle.com/saas-fusion-app-performance/how-to-set-stack-size-to-overcome-javalangstackoverflowerror")
           


import scala.annotation.tailrec


defined function fibonacci_tail_recursive
defined function fibonacci_not_tail_recursive
res2_3: BigInt = 93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000
res2_4: BigInt = 93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000
res2_5: BigInt = 33473205095971448369154760940714864779127732238104548077301003219901680221443656416973812310719169308798480438190208299893616384743066693742630572845363784038325756282123359987268244078235972356040853854441373383753568565536371168327405166076155165921406156075461294201790567479665498629242220022541553510718159801615476451810616674970217996537474972541139338191638823500630307644256874857271394651081909874909643486268589229807870031031008962861154553979911612940652327396971497211031261142860733793509687837

#### Use assertions [http://wiki.c2.com/?UseAssertions](http://wiki.c2.com/?UseAssertions)

## Currying and higher order functions

In [36]:
def operationOnInterval(op: (Int, Int) => Int, init: Int)(f: Int=>Int)(i: Int, j: Int): Int = {
  if (i > j) init
  else op(f(i), operationOnInterval(op, init)(f)(i + 1, j))
}

def product(f: Int => Int)(i: Int, j: Int) =
  operationOnInterval((i:Int, j:Int) => i * j, 1)(f)(i, j)

assert(product(k => k * k)(1,4) == 576)
assert(product(k => k)(1,4) == 24)

def isCloseEnough(x: Double, y: Double, tollerance: Double = 0.0001): Boolean = {
  Math.abs(x - y) / y < tollerance
}
assert(! isCloseEnough(1, 1.001, 0.0005))
assert(isCloseEnough(1, 1.001, 0.01))

@tailrec def fixedPoint(f: Double => Double)(init: Double): Double = {
  val next = f(init)
  println(next)
  if(isCloseEnough(next, init)) next
  else fixedPoint(f)(next)
}

fixedPoint(x => 1 + x / 2)(1)

// We can use averageDump whenever we use f, as returned function takes a param (currying)
def averageDump(f: Double => Double)(x: Double): Double = (f(x) + x) / 2

// here we use a fixed point method to define square root
// sqrt is y that y * y = x, so y = x / y - if we find a point where y = x/y, we are close :)

def sqrt(x: Double) = fixedPoint(averageDump((y:Double) => x / y))(1)

sqrt(2)

assert(isCloseEnough(sqrt(2), Math.sqrt(2)))

1.5
1.75
1.875
1.9375
1.96875
1.984375
1.9921875
1.99609375
1.998046875
1.9990234375
1.99951171875
1.999755859375
1.9998779296875
1.5
1.4166666666666665
1.4142156862745097
1.4142135623746899
1.5
1.4166666666666665
1.4142156862745097
1.4142135623746899


defined function operationOnInterval
defined function product
defined function isCloseEnough
defined function fixedPoint
res35_8: Double = 1.9998779296875
defined function averageDump
defined function sqrt
res35_11: Double = 1.4142135623746899

#### Exercise

 1. Define product method (tail recursive)
 1. Define factorial using product

## Classes

#### Use `require` to validate input

In [50]:
class Rational(x: Int, y: Int) {
    require(y != 0, "Denominator cannot be 0")
    
    private def gcd(a: Int, b: Int): Int = {
        if (b == 0) a else gcd(b, a % b)
    }
    val g = gcd(x, y)
    val numer = x / g
    val denom = y / g
    
    def this(x: Int) = this(x, 1)
    
    def + (that: Rational) =
        new Rational(
            this.numer * that.denom + that.numer * this.denom,
            this.denom * that.denom
        )
    
    def < (that: Rational) = this.numer * that.denom < that.numer * this.denom
    
    def * (i: Int) = new Rational(numer * i, denom)
    
    def max (that: Rational) = if(this < that) that else this
    
    def unary_- = new Rational(-numer, denom)
    
    def - (other: Rational) = this + -other
    
    override def toString = s"$numer / $denom"
}

// new Rational(1,0) // throws an IllegalArgumentException

val one = new Rational(1)
val two = new Rational(2)
val twoThirds = new Rational(2,3)
val half = new Rational(1,2)

// methods as infix operators
two + twoThirds
half + half
half - half

two < half


// Operator precedence in scala is predefiend 
// http://scala-lang.org/files/archive/spec/2.11/06-expressions.html#infix-operations
two max half * 2 + half

defined class Rational
one: wrapper.wrapper.Rational = 1 / 1
two: wrapper.wrapper.Rational = 2 / 1
twoThirds: wrapper.wrapper.Rational = 2 / 3
half: wrapper.wrapper.Rational = 1 / 2
res49_5: wrapper.wrapper.Rational = 8 / 3
res49_6: wrapper.wrapper.Rational = 1 / 1
res49_7: wrapper.wrapper.Rational = 0 / 1
res49_8: Boolean = false
res49_9: wrapper.wrapper.Rational = 2 / 1